TO TRAIN A CROP YIELD PREDICTION MODEL

IMPORT HEADER

In [11]:
import numpy as np
import pandas as pd
import pickle
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

LOAD DATASET

In [12]:
file_path = "Crop_Yield_Data_India_With_Crop_Names.csv"
crop_yield_data = pd.read_csv(file_path)


FIX COLUMN NAMES

In [13]:
crop_yield_data.rename(columns={'Crop Name': 'Crop', 'precipitati': 'precipitation', 'temperatu': 'temperature_2m'}, inplace=True)

DROP UNNECESSARY COLUMNS

In [14]:
crop_yield_data.drop(columns=["system:index", ".geo"], errors='ignore', inplace=True)

CONVERT TEMPERATURE FROM KELVIN TO CELSIUS

In [15]:
crop_yield_data["temperature_2m"] = crop_yield_data["temperature_2m"] - 273.15


Normalize NDVI if Required

In [16]:
if crop_yield_data["NDVI"].max() > 1 or crop_yield_data["NDVI"].min() < -1:
    crop_yield_data["NDVI"] = crop_yield_data["NDVI"] / 10000  

Encode Categorical Data

In [17]:
state_encoder = LabelEncoder()
crop_encoder = LabelEncoder()
crop_yield_data["State"] = state_encoder.fit_transform(crop_yield_data["State"])
crop_yield_data["Crop"] = crop_encoder.fit_transform(crop_yield_data["Crop"])

Define Features & Target

In [18]:
feature_columns = ["State", "Crop", "precipitation", "temperature_2m"]  
X = crop_yield_data[feature_columns]  
y = crop_yield_data["NDVI"]

Feature Scaling

In [19]:
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

Train Multiple Models

In [20]:
models = {
    "Random Forest": RandomForestRegressor(n_estimators=100, random_state=42),
    "Linear Regression": LinearRegression(),
    "Support Vector Machine": SVR(kernel='rbf'),
    "Gradient Boosting": GradientBoostingRegressor(n_estimators=100, random_state=42)
}

model_scores = {}
for model_name, model in models.items():
    model.fit(X_scaled, y)
    y_pred = model.predict(X_scaled)
    r2 = r2_score(y, y_pred) * 100
    mae = mean_absolute_error(y, y_pred)
    mse = mean_squared_error(y, y_pred)
    model_scores[model_name] = r2
    print(f"{model_name} Model Performance:")
    print(f"✅ R2 Score: {r2:.2f}%")
    print(f"✅ MAE: {mae:.4f}")
    print(f"✅ MSE: {mse:.4f}")
    print("====================================")


Random Forest Model Performance:
✅ R2 Score: 93.14%
✅ MAE: 0.0358
✅ MSE: 0.0026
Linear Regression Model Performance:
✅ R2 Score: 24.91%
✅ MAE: 0.1411
✅ MSE: 0.0289
Support Vector Machine Model Performance:
✅ R2 Score: 62.31%
✅ MAE: 0.0929
✅ MSE: 0.0145
Gradient Boosting Model Performance:
✅ R2 Score: 65.34%
✅ MAE: 0.0890
✅ MSE: 0.0133


Select Best Model

In [21]:
best_model_name = max(model_scores, key=model_scores.get)
best_model = models[best_model_name]
print(f"🎯 Best Model: {best_model_name} with {model_scores[best_model_name]:.2f}% Accuracy")

🎯 Best Model: Random Forest with 93.14% Accuracy


Save Best Model & Encoders

In [22]:
pickle.dump(best_model, open('model.pkl', 'wb'))
pickle.dump(scaler, open('scaler.pkl', 'wb'))
pickle.dump(state_encoder, open('state_encoder.pkl', 'wb'))
pickle.dump(crop_encoder, open('crop_encoder.pkl', 'wb'))

=================== ✅ PREDICT FUNCTION (NO CROP INPUT REQUIRED) ===================

In [23]:
def predict_yield(state_name):
    model = pickle.load(open('model.pkl', 'rb'))
    scaler = pickle.load(open('scaler.pkl', 'rb'))
    state_encoder = pickle.load(open('state_encoder.pkl', 'rb'))
    crop_encoder = pickle.load(open('crop_encoder.pkl', 'rb'))
    
    if state_name not in state_encoder.classes_:
        print(f"❌ Error: State '{state_name}' not found in dataset!")
        return

    state_encoded = state_encoder.transform([state_name])[0]
    state_data = crop_yield_data[crop_yield_data["State"] == state_encoded]
    
    if state_data.empty:
        print(f"❌ No data found for {state_name}!")
        return
    
    most_common_crop_encoded = state_data["Crop"].mode()[0]
    most_common_crop = crop_encoder.inverse_transform([most_common_crop_encoded])[0]
    
    state_crop_data = state_data[state_data["Crop"] == most_common_crop_encoded]
    avg_precipitation = state_crop_data["precipitation"].mean()
    avg_temperature_celsius = state_crop_data["temperature_2m"].mean()
    
    sample_input = pd.DataFrame([[state_encoded, most_common_crop_encoded, avg_precipitation, avg_temperature_celsius]], 
                                columns=feature_columns)
    
    sample_input_scaled = scaler.transform(sample_input)  
    predicted_ndvi = model.predict(sample_input_scaled)[0]
    predicted_ndvi = max(-1, min(1, predicted_ndvi))
    crop_yield_tons = (predicted_ndvi + 1) * 3  
    
    print(f"\n🌾 Predicted Data for {state_name}:")
    print(f"- Crop: {most_common_crop}")
    print(f"- Precipitation: {avg_precipitation:.2f} mm")
    print(f"- Temperature: {avg_temperature_celsius:.2f}°C")
    print(f"- NDVI: {predicted_ndvi:.4f}")  
    print(f"- Estimated Crop Yield: {crop_yield_tons:.2f} tons per hectare")


✅ Example Usage

In [24]:

predict_yield("Punjab")
predict_yield("Kerala")
predict_yield("Karnataka")


🌾 Predicted Data for Punjab:
- Crop: Wheat
- Precipitation: 2.58 mm
- Temperature: 23.42°C
- NDVI: 0.7893
- Estimated Crop Yield: 5.37 tons per hectare

🌾 Predicted Data for Kerala:
- Crop: Coconut
- Precipitation: 6.85 mm
- Temperature: 25.25°C
- NDVI: 0.8200
- Estimated Crop Yield: 5.46 tons per hectare

🌾 Predicted Data for Karnataka:
- Crop: Sugarcane
- Precipitation: 2.50 mm
- Temperature: 25.20°C
- NDVI: 0.5227
- Estimated Crop Yield: 4.57 tons per hectare


In [25]:
predict_yield("Tamil Nadu")




🌾 Predicted Data for Tamil Nadu:
- Crop: Rice
- Precipitation: 3.51 mm
- Temperature: 26.87°C
- NDVI: 0.5482
- Estimated Crop Yield: 4.64 tons per hectare


In [10]:

predict_yield("West Bengal")
predict_yield("Madhya Pradesh")


🌾 Predicted Data for West Bengal:
- Crop: Rice
- Precipitation: 4.19 mm
- Temperature: 25.89°C
- NDVI: 0.5595
- Estimated Crop Yield: 4.68 tons per hectare

🌾 Predicted Data for Madhya Pradesh:
- Crop: Wheat
- Precipitation: 2.89 mm
- Temperature: 24.94°C
- NDVI: 0.5298
- Estimated Crop Yield: 4.59 tons per hectare
